In [1]:
from imports import *
import staircase as sc
%matplotlib inline
import operator
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv(r"../data/hotel_stays.csv", parse_dates=['check_in', 'check_out'], dayfirst=True)

#df = df_cc.loc[df_cc.call_type == 'Type 1']

df.head()

,check_in,check_out,adults,children,reserved_room_type,assigned_room_type
0,2015-12-08 15:33:00,2016-01-05 09:56:00,2,0,E,E
1,2015-12-17 14:19:00,2016-01-05 09:04:00,2,0,E,E
2,2015-12-23 14:02:00,2016-01-02 06:58:00,2,0,E,E
3,2015-12-24 15:54:00,2016-01-01 08:48:00,2,0,D,D
4,2015-12-26 15:33:00,2016-01-01 09:59:00,2,2,G,G


In [3]:
#%%timeit -r 3 -n 3
hotel_stays = Steps.read_dataframe(df,'check_in', 'check_out')
#aborted_type1 = Steps.read_dataframe(df,'arrived','aborted')
#duration_type1 = Steps.read_dataframe(df,'answered','complete')

OutOfMemoryError: Out of memory allocating 218,112 bytes (allocated so far: 1,397,661,696 bytes).

In [ ]:
timespan = {'lbound':pd.Timestamp(2020,2,15,10),'ubound':pd.Timestamp(2020,2,16)}
ax = hotel_stays.plot(color='g',size=(36,6))
(hotel_stays>100).plot(ax=ax,color='r')

#aborted_type1.plot(ax=ax,color='r')
#duration_type1.plot(ax=ax,color='blue')